In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME                     PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
lebron-the-cluster-c3e8  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
# !pip install gensim

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib
from nltk.util import ngrams
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
from nltk.stem import WordNetLemmatizer
from math import log10

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'lebron_the_bucket_208627935' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

# Getting the data

In [6]:
# adding our python module to the cluster
%cd -q /home/dataproc
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

# Building an inverted index

In [6]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

stemmer = PorterStemmer()
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became",  "make",
                    "accordingly", "hence", "namely", "therefore", "thus",
                    "consequently",
                    "meanwhile",
                    "accordingly",
                    "likewise",
                    "similarly",
                    "notwithstanding",
                    "nonetheless",
                    "despite",
                    "whereas",
                    "furthermore",
                    "moreover",
                    "nevertheless",
                    "although",
                    "notably",
                    "notwithstanding",
                    "nonetheless",
                    "despite",
                    "whereas",
                    "furthermore",
                    "moreover",
                    "notably", "hence"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def tokenize_text(text, biagram, stem):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [tok for tok in tokens if tok not in all_stopwords]
    if stem:
        tokens = [stemmer.stem(token) for token in tokens]   
    if biagram:
        tokens = list(ngrams(tokens, 2))
        tokens = [' '.join(bigram) for bigram in tokens]   
    return tokens

def word_count(tokens_of_text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
    '''
    counter = Counter(tokens_of_text) 
    return [(token, (id, count)) for token, count in counter.items() if count > 10]

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
    Returns:
    --------
    list of tuples
      A sorted posting list.
    '''
    return sorted(unsorted_pl, key=lambda x: x[0])

# def add_tfIDF(postingList):
#     word_freq = len(postingList)
#     newPost = []
#     for post in postingList:
#         newPost.append((post[0], post[1]*log10(SIZE_OF_WIKI / word_freq)))
#     return newPost

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    # Count the document frequency for each token
    return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings, folder_name):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
    '''
    # posting in postings is word and posting list tuple
    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), x))
    # Group postings by bucket ID
    grouped_postings = bucketed_postings.groupByKey()
    # Write each bucket's postings to disk and collect locations
    posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, folder_name, bucket_name))

    return posting_locations

In [7]:
def create_posting_locs(doc_text_pairs, folder_name, Biagrm, Stem, filter_thershold):
    # word counts map
    print(f'started to create posting locs in folder_name: {folder_name}')
    tokenized_text_pairs = doc_text_pairs.map(lambda text_id_pair: (tokenize_text(text_id_pair[0], Biagrm, Stem), text_id_pair[1]))
    print(f'finished tokenizing in folder_name: {folder_name}')
    word_counts = tokenized_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
    print(f'finished word countes in folder_name: {folder_name}')
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    print(f'created postings in folder_name: {folder_name}')
    postings_filtered = postings.filter(lambda x: len(x[1]) > filter_thershold)
    print(f'filtered postings in folder_name: {folder_name}')
    # partition posting lists and write out
    partition_postings_and_write(postings_filtered, folder_name)
    print(f'finished writing to disk in folder: {folder_name}')
    doc_count_res = tokenized_text_pairs.map(lambda tok_text_pair: (tok_text_pair[1], len(tok_text_pair[0])))
    doc_len_map = doc_count_res.collectAsMap()
    print(f'finished creating doc_len_map in folder: {folder_name}')
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    print(f'finished w2df in folder: {folder_name}')
    return w2df_dict, doc_len_map
    

In [8]:
def read_posting_locs(folder_name):
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=f'{folder_name}'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
            for k, v in posting_locs.items():
                super_posting_locs[k].extend(v)
    return super_posting_locs

# Putting it all together

In [9]:
def create_posting_map(super_posting_locs, w2df_dict, inverted_name, doc_len_map):
    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    #doc len dic
    inverted.dl = doc_len_map
    # write the global stats out
    inverted.write_index('.', inverted_name)
    # upload to gs
    index_src = f"{inverted_name}.pkl"
    index_dst = f'gs://{bucket_name}/inverted_indices/{index_src}'
    !gsutil cp $index_src $index_dst
    return inverted

In [10]:
def create_invertedIndex(doc_text_pairs, folder_name, Biagram, Stem, filter_thershold):
    w2df_dict, doc_len_map = create_posting_locs(doc_text_pairs, folder_name, Biagram, Stem, filter_thershold)
    super_posting_locs = read_posting_locs(folder_name)
    return create_posting_map(super_posting_locs, w2df_dict, folder_name, doc_len_map)

In [ ]:
STEMMING_BODY_FOLDER = 'inverted_text_with_stemming'
STEMMING_TITLE_FOLDER = 'inverted_title_with_stemming'
NO_STEM_TITLE_FOLDER = 'inverted_title_no_stem'
NO_STEM_ANCHOR_FOLDER = 'inverted_anchor_no_stem'
STEM_ANCHOR_FOLDER = 'inverted_anchor_stem'
ANCHOR_RELATION = 'anchor_relations'

parquetFile = spark.read.parquet(*paths)
text_data = parquetFile.select("text", "id").rdd  
inverted1 = create_invertedIndex(text_data, f'{STEMMING_BODY_FOLDER}', False, True, 50)

started to create posting locs in folder_name: inverted_text_with_stemming
finished tokenizing in folder_name: inverted_text_with_stemming
finished word countes in folder_name: inverted_text_with_stemming
created postings in folder_name: inverted_text_with_stemming
filtered postings in folder_name: inverted_text_with_stemming


In [ ]:
text_data = parquetFile.select("title", "id").rdd  
inverted_no_stem_title = create_invertedIndex(text_data, f'{NO_STEM_TITLE_FOLDER}', False, False, 20)

In [ ]:
title_data = parquetFile.select("title", "id").rdd  
inverted1 = create_invertedIndex(text_data, f'{STEMMING_TITLE_FOLDER}', False, True, 20)

In [ ]:
def return_ids_from_anchor_list(anchot_text):
    ids_list = []
    for tup in anchot_text:
        ids_list.append(tup[0])
    return ids_list


def return_text_from_anchor_list(anchot_text):
    text = ''
    for tup in anchot_text:
        text = text + ' ' + tup[1]
    return text


In [ ]:
parquetFile = spark.read.parquet(*paths)
anchor_data = parquetFile.select("anchor_text", "id").rdd 

anchor_text = anchor_data.map(lambda anchor_id_pair: (return_text_from_anchor_list(anchor_id_pair[0]), anchor_id_pair[1]))
create_invertedIndex(anchor_text, f'{STEM_ANCHOR_FOLDER}', False, True, 20)

In [ ]:
# print(anchor_relations)
# client = storage.Client()

# # Get the bucket
# bucket = client.bucket(bucket_name)

# anchor_relations_dic = anchor_relations.collectAsMap()
# data = pickle.dumps(anchor_relations_dic)
# blob = bucket.blob("anchor_relations.pkl")
# blob.upload_from_string(data)

# parquetFile = spark.read.parquet(*paths)
# anchor_data = parquetFile.select("anchor_text", "id").rdd 

# anchor_text = anchor_data.map(lambda anchor_id_pair: (return_text_from_anchor_list(anchor_id_pair[0]), anchor_id_pair[1]))
# create_invertedIndex(anchor_text, f'{NO_STEM_ANCHOR_FOLDER}', False, False,  20)

# parquetFile = spark.read.parquet(*paths)
# anchor_data = parquetFile.select("anchor_text", "id").rdd 
# anchor_relations = anchor_data.map(lambda anchor_id_pair: (anchor_id_pair[1], return_ids_from_anchor_list(anchor_id_pair[0])))

# PageRank

In [ ]:
def generate_graph(pages):
    vertices = pages.flatMap(lambda page: [(page[0], )] + [(link[0], ) for link in page[1]])

    edges = pages.flatMap(lambda page: [(page[0], link[0]) for link in page[1]])

    edges = edges.distinct()
    vertices = vertices.distinct()

    return edges, vertices

In [ ]:
parquetFile = spark.read.parquet(*paths)
pages_links = parquetFile.select("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr_full_run', compression="gzip")
pr.show()

# PageViews

In [ ]:
# creating page views dictionary and saving it to bucket
from pathlib import Path

pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_stem = p.stem
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
    pickle.dump(wid2pv, f)

In [ ]:
# Serialize the object using pickle
data = pickle.dumps(wid2pv)

client = storage.Client()

# Get the bucket
bucket = client.bucket(bucket_name)

# Create a new GCS object with the serialized data
blob = bucket.blob("wid2pv.pkl")
blob.upload_from_string(data)

In [ ]:
# Save doc id to title dict
text_data = parquetFile.select("id", "title").rdd
titles_dict = text_data.collectAsMap()
data = pickle.dumps(titles_dict)
blob = bucket.blob("id2title.pkl")
blob.upload_from_string(data)

###### MOST COMMON

In [ ]:
# def create_new_inverted(old_inverted, most_common_map, inverted_name):
#     inverted = InvertedIndex()
#     inverted.posting_locs = old_inverted.posting_locs
#     inverted.df = old_inverted.df
#     inverted.dl = old_inverted.dl
#     inverted.mcw = most_common_map
#     inverted.write_index('.', inverted_name)
#     index_src = f"{inverted_name}.pkl"
#     index_dst = f'gs://{bucket_name}/inverted_indices/{index_src}'
#     !gsutil cp $index_src $index_dst
#     return inverted

In [ ]:
# folder_path = 'inverted_indices'
# def fetch_curr_inverted_index(index_name):
#     pickle_index = InvertedIndex.read_index(folder_path, index_name, bucket_name)
#     return pickle_index

In [ ]:
# STEMMING_BODY_FOLDER = 'inverted_text_with_stemming'
# BIGRAM_BODY_FOLDER = 'inverted_text_with_bigram'
# STEMMING_TITLE_FOLDER = 'inverted_title_with_stemming'
# BIGRAM_TITLE_FOLDER = 'inverted_title_with_bigram'
# STEMMING_MOST_COMMON_INVERTED_TEXT = 'inverted_text_with_stemming_two'
# BIGRAM_MOST_COMMON_INVERTED_TEXT = 'inverted_text_with_bigram_two'
# STEMMING_MOST_COMMON_INVERTED_TITLE = 'inverted_title_with_stemming_two'
# BIGRAM_MOST_COMMON_INVERTED_TITLE = 'inverted_title_with_bigram_two'


# def find_most_common_word(text, biagram):
#     tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#     tokens = [tok for tok in tokens if tok not in all_stopwords]
#     tokens = [stemmer.stem(token) for token in tokens]
#     if biagram:
#         tokens = list(ngrams(tokens, 2))
#         tokens = [' '.join(bigram) for bigram in tokens]   
#     if tokens:
#         most_common_token = Counter(tokens).most_common(1)[0][1]
#         return most_common_token
#     return 0 

# #Save doc id to most common word to title dict
# doc_data = parquetFile.select("id", "text").rdd  
# print("finished reading data")
# doc_to_most_common = doc_data.mapValues(lambda text: find_most_common_word(text, False))
# print("mapping doc to most common")
# titles_dict = doc_to_most_common.collectAsMap()
# print("converted to map")
# curr_inverted = fetch_curr_inverted_index(STEMMING_BODY_FOLDER)
# new_inverted = create_new_inverted(curr_inverted, titles_dict, STEMMING_MOST_COMMON_INVERTED_TEXT)
# print(len(new_inverted.mcw))

In [ ]:
# STEMMING_MOST_COMMON_INVERTED_TITLE = 'inverted_title_with_stemming_two'
# doc_data = parquetFile.select("id", "title").rdd  
# doc_to_most_common = doc_data.mapValues(lambda text: find_most_common_word(text, False))
# titles_dict = doc_to_most_common.collectAsMap()
# curr_inverted = fetch_curr_inverted_index(STEMMING_TITLE_FOLDER)
# new inverted = create_new_inverted(curr_inverted, titles_dict, STEMMING_MOST_COMMON_INVERTED_TITLE)

In [ ]:
# doc_data = parquetFile.select("id", "text").rdd  
# doc_to_most_common = doc_data.mapValues(lambda text: find_most_common_word(text, True))
# titles_dict = doc_to_most_common.collectAsMap()
# curr_inverted = fetch_curr_inverted_index(BIGRAM_BODY_FOLDER)
# new inverted = create_new_inverted(curr_inverted, titles_dict, BIGRAM_MOST_COMMON_INVERTED_TEXT)

In [ ]:
# doc_data = parquetFile.select("id", "title").rdd  
# doc_to_most_common = doc_data.mapValues(lambda text: find_most_common_word(text, True))
# titles_dict = doc_to_most_common.collectAsMap()
# curr_inverted = fetch_curr_inverted_index(BIGRAM_TITLE_FOLDER)
# new inverted = create_new_inverted(curr_inverted, titles_dict, BIGRAM_MOST_COMMON_INVERTED_TITLE)

In [7]:
old_bucket = 'tfidf_bucket_318437159'
STEMMING_TITLE_FOLDER = 'inverted_title_with_stemming'
index_src = f"{STEMMING_TITLE_FOLDER}.pkl"

index_src = f'gs://{old_bucket}/inverted_indices/{index_src}'
index_dst = f'gs://{bucket_name}/inverted_indices/{index_src}'
!gsutil cp $index_src $index_dst

AccessDeniedException: 403 There is an account problem for the requested project.
